In [17]:
import urllib.parse

CLIENT_ID = '5ptlhyjetbti68xyl0p5clqd77idkc'
REDIRECT_URI = 'http://localhost:8080'
SCOPES = 'chat:read chat:edit user:write:chat user:bot channel:bot'

authorize_url = f'https://id.twitch.tv/oauth2/authorize?response_type=code&client_id={CLIENT_ID}&redirect_uri={urllib.parse.quote(REDIRECT_URI)}&scope={urllib.parse.quote(SCOPES)}'

print(f'Go to the following URL to authorize your application: {authorize_url}')

result = 'http://localhost:8080/?code=47osmls74cibfxoq3p3n3yslb43ils&scope=chat%3Aread+chat%3Aedit+user%3Awrite%3Achat+user%3Abot+channel%3Abot'
code = 'kz338m3z761ttuyd9dq5pdvt954kl3'

Go to the following URL to authorize your application: https://id.twitch.tv/oauth2/authorize?response_type=code&client_id=5ptlhyjetbti68xyl0p5clqd77idkc&redirect_uri=http%3A//localhost%3A8080&scope=chat%3Aread%20chat%3Aedit%20user%3Awrite%3Achat%20user%3Abot%20channel%3Abot


In [18]:
import requests

def exchange_code_for_token(auth_code):
    url = 'https://id.twitch.tv/oauth2/token'
    data = {
        'client_id': '5ptlhyjetbti68xyl0p5clqd77idkc',
        'client_secret': 'dmt8imlq3gevbhu7ct0k1nbr3cayrn',
        'code': auth_code,
        'grant_type': 'authorization_code',
        'redirect_uri': REDIRECT_URI
    }
    response = requests.post(url, data=data)
    return response.json()

# After user authorizes and is redirected back to your redirect URI, extract the code from the URL and exchange it for tokens
auth_code = '47osmls74cibfxoq3p3n3yslb43ils'
tokens = exchange_code_for_token(auth_code)
access_token = tokens['access_token']
refresh_token = tokens['refresh_token']


In [19]:
tokens

{'access_token': 'lqmxercufphuasp7ieh4p5ziqmtcvv',
 'expires_in': 15064,
 'refresh_token': '1btkdtjkara1txh3px618let71wdqheemffhz7a1s64zd53b89',
 'scope': ['channel:bot',
  'chat:edit',
  'chat:read',
  'user:bot',
  'user:write:chat'],
 'token_type': 'bearer'}

In [4]:
headers = {
    'Authorization': f'Bearer {access_token}',
    'Client-ID': CLIENT_ID
}

response = requests.get('https://api.twitch.tv/helix/users', headers=headers)
print(response.json())

{'data': [{'id': '38823980', 'login': 'jweewee', 'display_name': 'jweewee', 'type': '', 'broadcaster_type': '', 'description': '', 'profile_image_url': 'https://static-cdn.jtvnw.net/user-default-pictures-uv/13e5fa74-defa-11e9-809c-784f43822e80-profile_image-300x300.png', 'offline_image_url': '', 'view_count': 0, 'created_at': '2012-12-28T13:56:09Z'}]}


In [ ]:
def refresh_access_token(refresh_token):
    url = 'https://id.twitch.tv/oauth2/token'
    data = {
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token,
        'client_id': CLIENT_ID,
        'client_secret': 'dmt8imlq3gevbhu7ct0k1nbr3cayrn'
    }
    response = requests.post(url, data=data)
    return response.json()

new_tokens = refresh_access_token(refresh_token)
access_token = new_tokens['access_token']
refresh_token = new_tokens['refresh_token']


In [5]:
import time
import json
import logging
import requests
from collections import Counter
from confluent_kafka import Consumer, KafkaError
from twitchio.ext import commands


import config

In [26]:
logging.basicConfig(filename='de.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Twitch API endpoint for sending messages
TWITCH_API_ENDPOINT = 'https://api.twitch.tv/helix/chat/messages'


class TwitchBot:
    def __init__(self, token, channel):
        self.token = token
        self.channel = channel
        logging.info(f"Bot setup!")


    def send_message(self, message):
        headers = {
            'Authorization': f'Bearer {self.token}',
            'Client-Id': CLIENT_ID,
            'Content-Type': 'application/json'
        }
        payload = {
            'channel': self.channel,
            'message': message
        }
        response = requests.post(TWITCH_API_ENDPOINT, headers=headers, json=payload)
        if response.status_code == 200:
            logging.info(f"Message sent to Twitch chat: {message}")
        else:
            logging.error(f"Failed to send message: {response.status_code} {response.text}")


class KafkaConsumer:
    def __init__(self, bot):
        self.bot = bot
        self.configuration = config.read_config()
        self.configuration["group.id"] = "python-group-1"
        self.configuration["auto.offset.reset"] = "earliest"
        self.consumer = Consumer(self.configuration)
        self.topic = "reddit_nba"

    def start_consuming(self):
        self.consumer.subscribe([self.topic])
        try:
            while True:
                msg = self.consumer.poll(1.0)
                if msg is not None and msg.error() is None:
                    key = msg.key().decode("utf-8")
                    value = msg.value().decode("utf-8")

                    comment_data = json.loads(value)
                    comment_body = comment_data['body']
                    logging.info(f"Consumed message from topic: {self.topic}: Key/Comment_id-> {key} Value-> Comment_data of id: {key}")

                    self.bot.send_message(comment_body)
        except KeyboardInterrupt:
            pass
        finally:
            self.consumer.close()
            logging.info("Consumer closed.")


def main():
    # access_token = "lqmxercufphuasp7ieh4p5ziqmtcvv"
    logging.info("Starting bot...")
    bot = TwitchBot(token=access_token, channel='jweewee')
    logging.info("Bot setup...")
    # kafka_consumer = KafkaConsumer(bot)
    # kafka_consumer.start_consuming()
    bot.send_message("Hello World!")

if __name__ == "__main__":
    main()